In [1]:
! export CUDA_VISIBLE_DEVICES=1,2

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from vllm import LLM, SamplingParams
import torch
from baichuan_generation_utils import build_chat_input
from vllm_model import Baichuan

/home/lixy/anaconda3/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-24 11:21:58,740	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
model_path = "pre_train_model/Baichuan2-7B-Chat"
llm = Baichuan(model_path)


In [4]:
tokenizer = AutoTokenizer.from_pretrained(
                        model_path,
                        trust_remote_code=True
                    )
model = LLM(model=model_path,
                            tokenizer=model_path,
                            tensor_parallel_size=2,     # 如果是多卡，可以自己把这个并行度设置为卡数N
                            trust_remote_code=True,
                            gpu_memory_utilization=0.8, # 可以根据gpu的利用率自己调整这个比例
                            dtype='half')

WARNING 11-24 11:22:01 config.py:1354] Casting torch.bfloat16 to torch.float16.


INFO 11-24 11:22:03 config.py:698] Defaulting to use mp for distributed inference
INFO 11-24 11:22:03 llm_engine.py:169] Initializing an LLM engine (v0.5.1) with config: model='pre_train_model/Baichuan2-7B-Chat', speculative_config=None, tokenizer='pre_train_model/Baichuan2-7B-Chat', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=pre_train_model/Baichuan2-7B-Chat, use_v2_block_manager=False, enable_prefix_caching=False)
WARNING 11-24 11:22:03 tokenize

In [5]:
prompts = ["吉利汽车座椅按摩","吉利汽车语音组手唤醒","自动驾驶功能介绍"]

In [6]:
batch_text = []
for q in prompts:
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {"role": "user", "content": q},
    ]
    chat, _ = build_chat_input(model, tokenizer, messages)
    tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, return_tensors="pt")
    batch_text.append(tokenized_chat)
# batch_input = torch.cat(batch_text, dim=0)
outputs = model.generate(batch_text)

AttributeError: 'LLM' object has no attribute 'generation_config'

In [19]:
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [22]:
outputs

[RequestOutput(request_id=1, prompt='[gMASK]sop<|system|>\n You are a helpful assistant.<|user|>\n 吉利汽车座椅按摩<|assistant|>', prompt_token_ids=[64790, 64792, 64790, 64792, 906, 31007, 13361, 31007, 30994, 13, 809, 383, 260, 6483, 9319, 30930, 64795, 30910, 13, 30910, 40457, 32031, 38555, 37972, 64796], prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' \n 吉利汽车座椅按摩功能可以带给您更加舒适的乘坐体验。', token_ids=(30910, 13, 30910, 40457, 32031, 38555, 37972, 31877, 31628, 39220, 55353, 31923, 47108, 34895, 32300, 31155), cumulative_logprob=-7.034255657988979, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1732412855.962891, last_token_time=1732412855.962891, first_scheduled_time=1732412855.9705887, first_token_time=1732412856.0352516, time_in_queue=0.007697582244873047, finished_time=1732412856.5612748), lora_request=None),
 RequestOutput(request_id=2, prompt='[gMASK]sop<|system|>\n You are a helpful assistant.<|user|>\n 吉利汽车语音组手唤醒<|as